# Querying from mta_data.db into Python via SQLAlchemy & data cleaning

In [2]:
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd

### Connecting Python to mta_data.db

In [3]:
#conda install numpy
#conda install -c intel mkl
#conda update mkl

In [4]:
engine = create_engine("sqlite:///mta_data.db")
#engine = create_engine("sqlite:///data/doctors.db")

In [5]:
insp = inspect(engine)
print(insp.get_table_names())

['error_entries', 'mta_data']


In [6]:
### Explore the database

In [ ]:
mta = pd.read_sql('SELECT * FROM mta_data;', engine)
mta

The combination of C/A, unit, SCP, and station define a unique turnstyle. Entry and exit counts for the turnstyle C/A A002, Unit R051, SCP 02-00-00, station 59 ST shows that the counts are cumulative. However, that for turnstyle C/A TRAM2, Unit R469, SCP 00-05-01, Station RIT-ROOSEVELT shows the same counts for entries and exits throughout the day. They are neither cumulative nor counts for the specific four hours as counts are unlikely to be exactly identical for all four hour slots in a day. We need to do a sanity check to ensure the data analyzed makes sense. 
First we will look at different subsets of the data.

### Four SQLAlchemy commands WHERE, AND, GROUPBY, HAVING

In [ ]:
#sqalc_create_table = pd.read_sql('CREATE TABLE [mta_data].error_entries (CA TEXT NOT NULL, UNIT TEXT NOT NULL, SCP TEXT NOT NULL, STATION TEXT NOT NULL, ENTRIES INTEGER NOT NULL, EXITS INTEGER NOT NULL);', engine)
#sqalc_create_table
                                 

In [ ]:
sqlalc_where_and = pd.read_sql('SELECT * FROM mta_data WHERE ENTRIES > 1000 AND DATE = 08/28/2021 LIMIT 5;', engine)
sqlalc_where_and

In [ ]:
sqlalc_groupby_having = pd.read_sql('SELECT max(EXITS) FROM mta_data GROUP BY DATE HAVING DATE = 09/06/2019 LIMIT 5;', engine)
sqlalc_groupby_having

### SUB-QUERY & COMMON TABLE EXPRESSION

In [ ]:
sqlalc_subquery = pd.read_sql('SELECT Station, AVG(Exits_Minus_100) AS Average FROM (SELECT Station, Exits - 100 AS Exits_Minus_100 FROM mta_data) AS mta_data_new GROUP BY Station;', engine)
sqlalc_subquery

In [ ]:
sqlalc_CTE = pd.read_sql('WITH mta_data_new AS (SELECT Station, Exits - 100 AS Exits_Minus_100 FROM mta_data) SELECT Name, AVG(Exits_Minus_100) AS Average FROM mta_data_new GROUP BY Station;', engine)
sqlalc_CTE

In [1]:
mta.info(verbose=True)

NameError: name 'mta' is not defined

Info shows non-null columns upto a default of 1690785 rows. We have more rows than that. Also, date and time are of type object or string. We need to add a new column of the date-time format. 

In [11]:
mta.isnull()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
3747730,False,False,False,False,False,False,False,False,False,False,False
3747731,False,False,False,False,False,False,False,False,False,False,False
3747732,False,False,False,False,False,False,False,False,False,False,False
3747733,False,False,False,False,False,False,False,False,False,False,False


## For the remainder of the analysis please see the Jupyter Notebook 'EDA Final code.ipynb'. The kernel kept dying, so I moved my analysis to a new notebook. 

We next check for white space in the columns, if any, and the strip it off the columns. 

In [12]:
#mta.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [13]:
#mta.columns = [column.strip() for column in mta.columns]
#mta.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [14]:
#mta.DATE.value_counts().sort_index().head(15)


08/28/2021    30021
08/29/2020    29454
08/29/2021    29754
08/30/2020    29523
08/30/2021    30738
08/31/2019    29375
08/31/2020    30101
08/31/2021    29865
09/01/2019    29243
09/01/2020    29422
09/01/2021    30159
09/02/2019    29283
09/02/2020    29499
09/02/2021    29664
09/03/2019    29290
Name: DATE, dtype: int64

This is the number of turnstiles on each day that we get information on. 

In [15]:
#import datetime

In [16]:
#mta["DATE_TIME"] = pd.to_datetime(mta.DATE + " " + mta.TIME, format = "%m/%d/%Y %H:%M:%S")
mta["DATE_TIME"] = pd.to_datetime(mta.DATE + " " + mta.TIME, format = "%m/%d/%Y %H:%M:%S")
#mta["DATE"] = pd.to_datetime(mta.DATE,format = "%m/%d/%Y")
#mta["TIME"] = pd.to_datetime(mta.TIME,format = "%H:%M:%S")                             

In [17]:
#mta.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,00:00:00,REGULAR,7625998,2609113,2021-08-28 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,04:00:00,REGULAR,7626014,2609117,2021-08-28 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,08:00:00,REGULAR,7626024,2609135,2021-08-28 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,12:00:00,REGULAR,7626086,2609191,2021-08-28 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/28/2021,16:00:00,REGULAR,7626206,2609221,2021-08-28 16:00:00


In [18]:
#mta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3747735 entries, 0 to 3747734
Data columns (total 12 columns):
 #   Column     Dtype         
---  ------     -----         
 0   C/A        object        
 1   UNIT       object        
 2   SCP        object        
 3   STATION    object        
 4   LINENAME   object        
 5   DIVISION   object        
 6   DATE       object        
 7   TIME       object        
 8   DESC       object        
 9   ENTRIES    int64         
 10  EXITS      int64         
 11  DATE_TIME  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 343.1+ MB


Checking that a particular turnstile for a specific date-time is unique. We would like to work with the exit data as a patron is likely to be more relaxed when she is exiting the train station than when she is in 'a hurry to catch the train'. 

In [19]:
#(mta
 #.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 #.EXITS.count()
 #.reset_index()
 #.sort_values("EXITS", ascending=False)).head(66)

,C/A,UNIT,SCP,STATION,DATE_TIME,EXITS
2367047,R107,R305,00-00-01,WTC-CORTLANDT,2019-10-07 01:00:00,2
3717608,S101,R070,00-00-02,ST. GEORGE,2021-09-03 12:00:00,2
666533,H039,R375,00-00-00,NEW LOTS,2020-09-22 05:00:00,2
3158946,R315,R406,01-00-00,PROSPECT AV,2020-10-05 05:00:00,2
3717595,S101,R070,00-00-02,ST. GEORGE,2021-09-01 08:00:00,2
...,...,...,...,...,...,...
2367804,R107,R305,00-00-02,WTC-CORTLANDT,2019-10-07 09:00:00,2
810019,N007A,R174,00-00-00,181 ST,2020-09-23 13:00:00,2
2367803,R107,R305,00-00-02,WTC-CORTLANDT,2019-10-07 05:00:00,2
2498465,R138,R293,00-02-01,34 ST-PENN STA,2020-10-12 14:00:00,1


However, it looks like there are 64 repeats for the exit data. At least 3 of those turnstiles are in Station WTC-CORTLANDT on October 7, 2019. SCP 00-00-02 can be seen to contribute to two of these. Let's take a look at the first one. 

In [20]:
#mask = ((mta["C/A"] == "R107") & 
#(mta["UNIT"] == "R305") & 
#(mta["SCP"] == "00-00-01") & 
#(mta["STATION"] == "WTC-CORTLANDT") &
#(mta["DATE_TIME"].dt.date == datetime.datetime(2019, 10, 7).date()))

#mta[mask].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
2644321,R107,R305,00-00-01,WTC-CORTLANDT,1,IRT,10/07/2019,01:00:00,REGULAR,153675,41152,2019-10-07 01:00:00
2644322,R107,R305,00-00-01,WTC-CORTLANDT,1,IRT,10/07/2019,01:00:00,RECOVR AUD,229502,34045,2019-10-07 01:00:00
2644323,R107,R305,00-00-01,WTC-CORTLANDT,1,IRT,10/07/2019,05:00:00,REGULAR,153676,41152,2019-10-07 05:00:00
2644324,R107,R305,00-00-01,WTC-CORTLANDT,1,IRT,10/07/2019,05:00:00,RECOVR AUD,229503,34046,2019-10-07 05:00:00
2644325,R107,R305,00-00-01,WTC-CORTLANDT,1,IRT,10/07/2019,09:00:00,REGULAR,153691,41163,2019-10-07 09:00:00


In this case, the RECOVR AUD values reflect that one person went through the turnstile between 1 and 5 am, and the regular values do not reflect that. If we work with the RECOVR AUD values then between 5 and 9 am, 7117 people went through the turnstile and as per the regular values, only 9 people did. Estimates assume **3-4 seconds per person** and the RECOVR AUD values imply 2 seconds per person. It is unlikely also because WTC-CORTLANDT is likely not a busy station. Lets check how many RECOVR AUD there are in our data. 

In [21]:
#mta.DESC.value_counts()

REGULAR       3732504
RECOVR AUD      15231
Name: DESC, dtype: int64

0.4% of the data is RECOVR AUD which is in the same range as 3% of the data in  the mta-pair-2-solution. 

In [22]:
#We will drop the Recovr Aud entries leading to the duplicate absurd values.
#mta.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                         # inplace=True, ascending=False)
#mta.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

In [23]:
# Check to see if that takes care of all duplicate values in our data subset
#(mta
 #.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 #.EXITS.count()
 #.reset_index()
 #.sort_values("EXITS", ascending=False)).head()

,C/A,UNIT,SCP,STATION,DATE_TIME,EXITS
0,A002,R051,02-00-00,59 ST,2019-08-31 00:00:00,1
2498439,R138,R293,00-02-01,34 ST-PENN STA,2020-10-08 06:00:00,1
2498441,R138,R293,00-02-01,34 ST-PENN STA,2020-10-08 14:00:00,1
2498442,R138,R293,00-02-01,34 ST-PENN STA,2020-10-08 18:00:00,1
2498443,R138,R293,00-02-01,34 ST-PENN STA,2020-10-08 22:00:00,1


It does! Although we plan to use the exit values only, we will hold onto the entries values just in case we need them later but drop the DESC values.

In [24]:
#mta = mta.drop(["DESC"], axis=1, errors="ignore")

Since the values are cumulative, the maximum EXITS for each day would be the last reading of the day. 

In [ ]:
#mta_daily = (mta.groupby(["C/A", "UNIT", "SCP", "STATION","DATE_TIME"], as_index = False).EXITS.first())

In [ ]:
mta_daily.head()
#mta_daily.info()

The turnstiles are the same as that for the output of MTA Analysis Part 1 for 2016 data, which analyzed entries. Broadly, it appears that turnstiles recording high number of daily entries also record high numbers of daily exits. 
These are cumulative entries. Now we will check the new daily entries for each day. Before that, we will change the date to datetime format as we have dates for multiple years in our dataset. 

In [ ]:
#mta["DATE"] = pd.to_datetime(mta.DATE,format = "%m/%d/%Y")


In [ ]:
#mta_daily[["PREV_DATE", "PREV_EXITS"]] = (mta
                                      #.groupby(["C/A", "UNIT", "SCP", "STATION"])[["DATE_TIME", "EXITS"]]
                                      #.shift(1))
#mta_daily.head()#

The earliest rows in the previous date column has missing values as expected. We will drop those values. 

In [ ]:
#mta_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [ ]:
#To get the non-cumulative extra values:
#(mta_daily["EXITS"] - mta_daily["PREV_EXITS"]).describe()

The minimum and the 25% values are negative which is absurd for number of exits. The max number is absurdly high. A couple of the entries were cumulative backwards. 

In [ ]:
#Checking for number of backward cumulatives
#mta_daily[mta_daily["EXITS"] < mta_daily["PREV_EXITS"]].shape

In [ ]:
#mta_daily[mta_daily["EXITS"] < mta_daily["PREV_EXITS"]].head()